In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

exp = Experiment(workspace=ws, name="Project_1")

run = exp.start_logging()

UserErrorException: UserErrorException:
	Message: We could not find config.json in: C:\Datos\Aprendiendo\Machine Learning Engineer with Microsoft Azure Nanodegree Program\2. Using Azure Machine Learning\Proyecto1\nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "We could not find config.json in: C:\\Datos\\Aprendiendo\\Machine Learning Engineer with Microsoft Azure Nanodegree Program\\2. Using Azure Machine Learning\\Proyecto1\\nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories."
    }
}

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster

cpu_cluster_name = "bm-lr-ml"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    ### YOUR CODE HERE ###
    "--C": uniform(0.1, 1), #Explorar valores de C en el espacio 0.1 a 1 
    "--max_iter": choice(100, 150, 200, 250) #Escoger iteraciones en el espacio 100 a 250
    }
)    

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1,
                     evaluation_interval = 2) 


if "training" not in os.listdir():
    os.mkdir("./training")

os.replace("train.py", "./training/train.py")
    
# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(
            source_directory = "./traning",
            compute_target = compute_target,
            entry_script = "train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                       primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                       primary_metric_name = "Accuracy",
                                       max_concurrent_runs = 3,
                                       max_total_runs = 30,
                                       policy = policy,
                                       estimator = est)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run <- exp.submit(hyperdrive_config, show_output = True)

In [ ]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

model_hd = best_run.register_model(model_name='model', model_path='outputs/model.joblib')

os.makedirs('outputs', exist_ok=True)
joblib.dump(model_hd, 'outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", separator=',', encoding='utf8')

mien = dataset.to_pandas_dataframe()

mien.head()

In [ ]:
import sys
sys.path.insert(1, './training')

from train import clean_data

# Use the clean_data function to clean your data.

x, y = clean_data(ds)

from sklearn_model_selection import train_test_split
import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=37) 

df_train = pd.concat([x_train, y_train], axis = 1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'Accuracy',
    training_data = x_train,
    label_column_name = 'y',
    n_cross_validations = 5)


In [2]:
# Submit your automl run

remote_run = experiment.submit(automl_config, show_output = False)

RunDetails(remote_run).show

In [1]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

model_aml = best_run.register_model(model_name='model', model_path='outputs/model.joblib')

joblib.dump(model_aml, 'outputs/model.joblib')

NameError: name 'remote_run' is not defined